# Exploratory Data Analysis : Google Store Revenue Dataset

**data dictionary**
https://support.google.com/analytics/answer/3437719?hl=en

In [ ]:
# import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import datetime
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from sklearn import preprocessing

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# import warnings

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

### Dataset Info

In [ ]:
# read the data
train_df = pd.read_pickle('2017_clean.pkl')

In [ ]:
#train_df.shape
print("Datasets contains %s of features, and %s of rows."%(train_df.shape[1],train_df.shape[0]))
print("Number of unique visitors in train set : ",train_df.fullVisitorId.nunique(), " out of rows : ",train_df.shape[0])

In [ ]:
train_df.head()

In [ ]:
# understand the quantity of misisng values
missing_values_count = train_df.isnull().sum().sort_values(ascending=False)
missing_values_per = round((missing_values_count/len(train_df)) * 100,2)
pd.concat([missing_values_count, missing_values_per], axis=1, keys = ['Quant Missing values', 'Percentage Missing values - %'])

* In terms of of total revenue, about 1.2%of customers is generating the total revenues
* More missing values related to traffic source, might need to dig more into it.
* Some of NA (missing values) also contains values based on data dictionary. EX: Binary (1 and Na is 0)

**Change dataset into right format**

In [ ]:
# library of datetime
from datetime import datetime

# This function is to extract date, weekdays, day,month and visitHour features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["_weekday"] = df['date'].dt.weekday #extracting week day
    df["_day"] = df['date'].dt.day # extracting day
    df["_month"] = df['date'].dt.month # extracting day
    df["_year"] = df['date'].dt.year # extracting day
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)

    return df #returning the df after the transformations
train_df = date_process(train_df) #calling the function that we created above

In [ ]:
#change dataset into right format, then we can also save some memory space
#fill missing values(0,1/True,False)
train_df['totals.bounces'] = train_df['totals.bounces'].fillna(0)
train_df['totals.newVisits'] =train_df['totals.newVisits'].fillna(0)
train_df['trafficSource.isTrueDirect'] =train_df['trafficSource.isTrueDirect'].fillna('False')
train_df['trafficSource.adwordsClickInfo.slot'] = train_df['trafficSource.adwordsClickInfo.slot'].fillna('None')
train_df['trafficSource.adwordsClickInfo.adNetworkType'] = train_df['trafficSource.adwordsClickInfo.adNetworkType'].fillna('Unknown')
#train_df['totals.timeOnSite'] = train_df['totals.timeOnSite'].fillna(0)
#train_df['totals.transactions'] = train_df['totals.transactions'].fillna(0)
#train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(0.0)
#train_df['totals.totalTransactionRevenue'] = train_df['totals.totalTransactionRevenue'].fillna(0)
train_df['trafficSource.adwordsClickInfo.page'] = train_df['trafficSource.adwordsClickInfo.page'].fillna(0)
train_df['totals.hits']=train_df['totals.hits'].fillna(0)
train_df.loc[train_df['geoNetwork.city'] == "(not set)", 'geoNetwork.city'] = np.nan
train_df['geoNetwork.city'].fillna("NaN", inplace=True)
#change format of dataset
train_df['channelGrouping']=train_df['channelGrouping'].astype('category')
train_df['device.browser']=train_df['device.browser'].astype('category')
train_df['device.operatingSystem']=train_df['device.operatingSystem'].astype('category')
train_df['device.isMobile']=train_df['device.isMobile'].astype('category')
train_df['device.deviceCategory']=train_df['device.deviceCategory'].astype('category')
train_df['geoNetwork.continent']=train_df['geoNetwork.continent'].astype('category')
train_df['geoNetwork.subContinent']=train_df['geoNetwork.subContinent'].astype('category')
train_df['geoNetwork.country']=train_df['geoNetwork.country'].astype('category')
train_df['geoNetwork.region']=train_df['geoNetwork.region'].astype('category')
train_df['geoNetwork.city']=train_df['geoNetwork.city'].astype('category')
train_df['geoNetwork.networkDomain']=train_df['geoNetwork.networkDomain'].astype('category')
train_df['totals.hits']=train_df['totals.hits'].astype('int')
train_df['totals.pageviews']=train_df['totals.pageviews'].astype('float')
train_df['totals.bounces']=train_df['totals.bounces'].astype('float')
train_df['totals.newVisits']=train_df['totals.newVisits'].astype('float')
train_df['totals.sessionQualityDim']=train_df['totals.sessionQualityDim'].astype('float')
train_df['totals.timeOnSite']=train_df['totals.timeOnSite'].astype('float')
train_df['totals.transactions']=train_df['totals.transactions'].astype('float')
train_df['totals.transactionRevenue']=train_df['totals.transactionRevenue'].astype('float')
train_df['totals.totalTransactionRevenue']=train_df['totals.totalTransactionRevenue'].astype('float')
train_df['trafficSource.campaign'] =train_df['trafficSource.campaign'].astype('category')
train_df['trafficSource.source']=train_df['trafficSource.source'].astype('category')
train_df['trafficSource.medium']=train_df['trafficSource.medium'].astype('category')
train_df['trafficSource.isTrueDirect']=train_df['trafficSource.isTrueDirect'].astype('category')
train_df['trafficSource.adwordsClickInfo.slot']=train_df['trafficSource.adwordsClickInfo.slot'].astype('category')
train_df['trafficSource.adwordsClickInfo.adNetworkType']=train_df['trafficSource.adwordsClickInfo.adNetworkType'].astype('category')
train_df['trafficSource.adwordsClickInfo.page']=train_df['trafficSource.adwordsClickInfo.page'].astype('int')

In [ ]:
train_df.info()

In [ ]:
train_df.head()

**Data Analysis**

**1) Transaction Revenue Distribution**

In [ ]:
# Revenue
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
ax = sns.distplot(np.log(train_df[train_df['totals.totalTransactionRevenue'] > 0]["totals.totalTransactionRevenue"] + 0.01), bins=40, kde=True)
ax.set_xlabel('Transaction RevenueLog', fontsize=5)
ax.set_ylabel('Distribuition', fontsize=5)
ax.set_title("Distribuition of Revenue Log", fontsize=10)

plt.subplot(1,2,2)
gdf = train_df.groupby("fullVisitorId")["totals.totalTransactionRevenue"].sum().reset_index()
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.totalTransactionRevenue"].values)))
plt.xlabel('Index', fontsize=5)
plt.ylabel('Revenue value', fontsize=5)
plt.title("Revenue Value Distribution", fontsize=10)

plt.show()

* log of transaction revenue distribution is normally distributed
* The second graph shows minorities of customers produce majorties of benefits

***ChannelGrouping Analysis***

In [ ]:
count = train_df['channelGrouping'].value_counts()

plt.figure(figsize=(10,5))
ax = sns.barplot(count.index, count.values, alpha=0.8)
ax.set_xlabel('Type of channel')
ax.set_ylabel('Frequency')
plt.show()

* Most our customers are coming from Organic Search, then the next channel is from direct.

The TOP 5 Grouping Channels represents 97% of customer units. Respectivelly:

* TOP 1 => Organic Search - 42.99%
* TOP 2 => Social - 24.39%
* TOP 3 => Direct - 15.42%
* TOP 4 => Referral - 11.89%
* TOP 5 => Paid Search - 2.55%

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='channelGrouping', y='totals.totalTransactionRevenue', data=train_df)

* Display actually has the most revenue even though most our customer is from Organic Search
* The second most is from direct, it is also the second most group channel that get more customers

#### 1) Device related columns

***device.browser Analysis***

In [ ]:
crosstab_eda = pd.crosstab(index=train_df['channelGrouping'], normalize=True,
                           # at this line, I am using the isin to select just the top 5 of browsers
                           columns=train_df[train_df['device.browser'].isin(train_df['device.browser']\
                                                                            .value_counts()[:5].index.values)]['device.browser'])

crosstab_eda.plot(kind="bar",    # select the bar to plot the count of categoricals
                 figsize=(14,7), # adjusting the size of graphs
                 stacked=True)   # code to unstack
plt.title("Channel Grouping % for which Browser", fontsize=20) # seting the title size
plt.xlabel("The Channel Grouping Name", fontsize=18) # seting the x label size
plt.ylabel("Count", fontsize=18) # seting the y label size
plt.xticks(rotation=0)
plt.show() # rendering

* Majority of users is from Chrome,and the second popular browser is Safari.

***device.isMobile Analysis***

In [ ]:
sns.barplot(x='device.isMobile', y='totals.totalTransactionRevenue', data=train_df)

***Date***

In [ ]:
sns.lineplot(x="_month", y="totals.totalTransactionRevenue",hue='device.deviceCategory',data=train_df)

In [ ]:
sns.barplot(x="_visitHour", y="totals.totalTransactionRevenue", data=train_df)

In [ ]:
sns.lineplot(x="_weekday", y="totals.totalTransactionRevenue", data=train_df)

In [ ]:
sns.countplot(x='_visitHour', hue='device.deviceCategory', data=train_df)

In [ ]:
df_date2=pd.get_dummies(train_df['_weekday'],prefix='weekday',drop_first=True)
#df_date2=df_date2.join(pd.get_dummies(df_clean['_visitHour'],prefix='hour',drop_first=True))

In [ ]:
plt.figure(figsize=(10,10))
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

# Create scaler: scaler
#scaler = StandardScaler()

# Create a PCA instance: pca
pca = PCA()

# Create pipeline: pipeline


# Fit the pipeline to 'samples'
pca.fit(df_date2)

# Plot the explained variances
features = range(pca.n_components_)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('PCA feature')
plt.ylabel('cumulative variance')
plt.xticks(features)
plt.show()
# PCA() i can tell him the number of the pca I wanna use

In [ ]:
pca.components_

In [ ]:
df_date3=pd.get_dummies(train_df['_month'],drop_first=True,prefix='month')

In [ ]:
plt.figure(figsize=(10,10))
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

# Create scaler: scaler
#scaler = StandardScaler()

# Create a PCA instance: pca
pca = PCA()

# Create pipeline: pipeline


# Fit the pipeline to 'samples'
pca.fit(df_date3)

# Plot the explained variances
features = range(pca.n_components_)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('PCA feature')
plt.ylabel('cumulative variance')
plt.xticks(features)
plt.show()
# PCA() i can tell him the number of the pca I wanna use

In [ ]:
df_date3=pd.get_dummies(train_df['_visitHour'],drop_first=True,prefix='hour')

In [ ]:
plt.figure(figsize=(10,10))
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

# Create scaler: scaler
#scaler = StandardScaler()

# Create a PCA instance: pca
pca = PCA()

# Create pipeline: pipeline


# Fit the pipeline to 'samples'
pca.fit(df_date3)

# Plot the explained variances
features = range(pca.n_components_)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('PCA feature')
plt.ylabel('cumulative variance')
plt.xticks(features)
plt.show()
# PCA() i can tell him the number of the pca I wanna use

* We try to use pca for date, but it does not work very well for dimension reduction
* Going with exploratory analysis results
   * Creating new feature isTuesday, because people tends to make more transactions on Tuesday
   * isApril becuase there is a spike when wen plot month with transactions revenue

***device.deviceCategory***

In [ ]:
sns.barplot(x="device.deviceCategory", y="totals.totalTransactionRevenue", data=train_df)

In [ ]:
plt.figure(figsize=(10,8))
# revenue
count = train_df.groupby(['device.browser'])['totals.transactionRevenue'].agg(['sum'])
#sns.barplot(x='device.isMobile', y='totals.totalTransactionRevenue', data=train_df)
count.sort_values(['sum'], ascending=False)[:5].plot(kind='bar', fontsize=12)

In [ ]:
# device browser
count = train_df.groupby('device.operatingSystem')['totals.transactionRevenue'].agg(['sum'])
count.sort_values(['sum'], ascending=False)[:5].plot(kind='bar', fontsize=12)

**2) GeoNetwork Columns**

***geoNetwork.continent***   
Only 6 groups, ready to use.

Thoughts:
1. Americas has most counts and non-zero revenue.
2. Asia and Europe have high counts but low non-zero revenue.
3. Mean revenue of Africa is high.

In [ ]:
# continent
cnt = train_df.groupby('geoNetwork.continent')['totals.totalTransactionRevenue'].agg(['size', 'count', 'mean'])
cnt.columns = ["count", "count of non-zero revenue", "mean"]
cnt = cnt.sort_values(by="count", ascending=False)
cnt

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(1,3,1)
plt.barh(cnt.index.values, cnt['count'])
plt.title("Continent - Revenue Count", fontsize=13)

plt.subplot(1,3,2)
plt.barh(cnt.index.values, cnt['count of non-zero revenue'])
plt.title("Continent - non-zero revenue", fontsize=13)

plt.subplot(1,3,3)
plt.barh(cnt.index.values, cnt['mean'])
plt.title("Continent - Revenue Mean", fontsize=13)

***geoNetwork.subContinent***   
About 20 groups, ready to use.
Similar pattern as continent.

In [ ]:
# subcontinent
scnt = train_df.groupby('geoNetwork.subContinent')['totals.totalTransactionRevenue'].agg(['size', 'count', 'mean'])
scnt.columns = ["count", "count of non-zero revenue", "mean"]
scnt = scnt.sort_values(by="count", ascending=False)
scnt.head()

***geoNetwork.country***   
* United State has only 45% of counts but over 95% of the non-zero revenue. That's interesting. Other countries almost generate no revenue except Canada. But Canada also only generate 174 pieces of non-zero revenue out of 30k counts of visit.

* We may consider drop this column and create a feature **is_US** (1 for country==US, 0 for country!=US).

In [ ]:
# country
ct = train_df.groupby('geoNetwork.country')['totals.totalTransactionRevenue'].agg(['size', 'count', 'mean'])
ct.columns = ["count", "count of non-zero revenue", "mean"]
ct = ct.sort_values(by="count", ascending=False)

ct.head()

In [ ]:
train_df[train_df['geoNetwork.country']=='United States']['totals.totalTransactionRevenue'].count()/train_df['totals.totalTransactionRevenue'].count()

In [ ]:
len(train_df[train_df['geoNetwork.country']=='United States']['totals.totalTransactionRevenue'])/len(train_df['totals.totalTransactionRevenue'])

In [ ]:
# plot top 10
ct_10 = ct.head(10)

plt.figure(figsize=(7,10))

plt.subplot(3,1,1)
plt.barh(ct_10.index.values, ct_10['count'])
plt.title("Country - Revenue Count", fontsize=13)

plt.subplot(3,1,2)
plt.barh(ct_10.index.values, ct_10['count of non-zero revenue'])
plt.title("Country - non-zero revenue", fontsize=13)

plt.subplot(3,1,3)
plt.barh(ct_10.index.values, ct_10['mean'])
plt.title("Country - Revenue Mean", fontsize=13)

***geoNetwork.region***   
* For non-zero revenue, region not available is nearly 40%. Among the remaining, California is the most and is above 30%.

In [ ]:
# city
city = train_df.groupby('geoNetwork.city')['totals.totalTransactionRevenue'].agg(['size', 'count', 'mean'])
city.columns = ["count", "count of non-zero revenue", "mean"]
city = city.sort_values(by="count", ascending=False)

city.head(20)

In [ ]:
# plot top 20
city_20 = city.head(20)

plt.figure(figsize=(5,20))

plt.subplot(3,1,1)
plt.barh(city_20.index.values, city_20['count'])
plt.title("Continent - Revenue Count", fontsize=13)

plt.subplot(3,1,2)
plt.barh(city_20.index.values, city_20['count of non-zero revenue'])
plt.title("Continent - non-zero revenue", fontsize=13)

plt.subplot(3,1,3)
plt.barh(city_20.index.values, city_20['mean'])
plt.title("Continent - Revenue Mean", fontsize=13)

***geoNetwork.networkDomain***   
* About 29K domains, including unknown.unknown and (not set). It is interesting to find out that (not set) has better revenue than unknown.unknown.
* Looks like this column is ready to use.

* Clean up networkDomain and just keep ending like edu,net,com

In [ ]:
train_df['domain'] = train_df['geoNetwork.networkDomain'].str.split('.')
train_df['domain'] = [i[-1] for i in train_df['domain']]

In [ ]:
# networkDomain
dm = train_df.groupby('domain')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
dm.columns = ["count", "count of non-zero revenue", "mean"]
dm = dm.sort_values(by="count", ascending=False)

dm.head()

In [ ]:
# plot top 20
dm20 = dm.head(20)

plt.figure(figsize=(5,20))

plt.subplot(3,1,1)
plt.barh(dm20.index.values, dm20['count'])
plt.title("Domain - Revenue Count", fontsize=13)

plt.subplot(3,1,2)
plt.barh(dm20.index.values, dm20['count of non-zero revenue'])
plt.title("Domain - non-zero revenue", fontsize=13)

plt.subplot(3,1,3)
plt.barh(dm20.index.values, dm20['mean'])
plt.title("Domain - Revenue Mean", fontsize=13)

***Totals Columns Analysis***

In [ ]:
columns=['totals.hits',
 'totals.pageviews',
 'totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.transactions',
 'totals.transactionRevenue',
 'totals.totalTransactionRevenue']
train_total=train_df[columns]

In [ ]:
g = sns.pairplot(train_total)

In [ ]:
def heatMap(df):
    #Create Correlation df
    corr = df.corr()
    #Plot figsize
    fig, ax = plt.subplots(figsize=(10, 10))
    #Generate Color Map
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
    #Generate Heat Map, allow annotations and place floats in map
    sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
    #Apply xticks
    plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
    plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
    plt.show()

In [ ]:
heatMap(train_total)

* I think we can drop either total hits or page views since they share almost same information about customers and they have same correlation with revenue

In [ ]:
#sns.jointplot(x=train_df['totals.hits'], y=train_df['totals.transactionRevenue'],color ='g')
g=sns.jointplot(x=np.log(train_df['totals.hits']), y=np.log(train_df['totals.totalTransactionRevenue']),kind="reg")

In [ ]:
g=sns.jointplot(x=np.log(train_df['totals.pageviews']), y=np.log(train_df['totals.totalTransactionRevenue']),kind="reg")

In [ ]:
g=sns.jointplot(train_df['totals.sessionQualityDim'], y=np.log(train_df['totals.totalTransactionRevenue']),kind="reg")

* Both page views and total hits is highly correlated to each other. After log transformation, with simple linear regressions. Both total hits and page views is positively related to log of transaction revenues.

In [ ]:
g=sns.jointplot(np.log(train_df['totals.timeOnSite']), y=np.log(train_df['totals.totalTransactionRevenue']),kind="reg")

In [ ]:
tmp = train_df.groupby('fullVisitorId').size().value_counts().to_frame().reset_index()
sum_ = tmp[tmp['index'].astype('int16') > 5][0].sum()
tmp = tmp.head(5).append(pd.DataFrame({'index': ['more than 5'], 0: [sum_]})).reset_index()
plt.figure(figsize=(10,4))
plt.title("Visits per user")
ax = sns.barplot(x=tmp['index'], y=tmp[0], palette='Blues_d')

***total.timeOnSite***

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(train_df['totals.timeOnSite'].shape[0]), np.sort((train_df['totals.timeOnSite'].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('timeonSite', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(train_df['totals.timeOnSite'], train_df['totals.totalTransactionRevenue'])
plt.xlabel('Timeonside', fontsize=12)
plt.ylabel('Revenue', fontsize=12)
plt.show()

***Traffic Source Analysis***

In [ ]:
traffic=['trafficSource.campaign',
 'trafficSource.source',
 'trafficSource.medium',
 'trafficSource.keyword',
 'trafficSource.referralPath',
 'trafficSource.isTrueDirect',
 'trafficSource.adContent',
 'totals.transactionRevenue',
 'totals.totalTransactionRevenue']

In [ ]:
traffic_df=train_df[traffic]

In [ ]:
train_df['trafficSource.campaign'].unique()

* I think we can deal with this later. It looks very mass

In [ ]:
train_df['trafficSource.source'].unique()

In [ ]:
# new = train_df['trafficSource.source'].str.split(".", n = 1, expand = True)
# train_df['trafficSource.source1']=new[0]
# train_df['trafficSource.source1']=new[1]

Might need to do some further cleaning

In [ ]:
ax = sns.barplot(x=train_df['trafficSource.medium'], y=train_df['totals.totalTransactionRevenue'])

In [ ]:
ax = sns.barplot(x=train_df['trafficSource.isTrueDirect'], y=train_df['totals.totalTransactionRevenue'])

In [ ]:
data=train_df.groupby(["trafficSource.keyword"])["totals.totalTransactionRevenue"].sum().sort_values(ascending=False)
#sns.barplot(x="trafficSource.keyword", y="totals.totalTransactionRevenue", data=train_df)
data[1:10]

In [ ]:
train_df['trafficSource.referralPath'].unique()

In [ ]:
train_df['trafficSource.adwordsClickInfo.page'].unique()

In [ ]:
 ax = sns.barplot(x="trafficSource.adwordsClickInfo.page", y="totals.totalTransactionRevenue", data=train_df)

In [ ]:
train_df['trafficSource.keyword'].unique()

In [ ]:
train_df['trafficSource.adContent'].value_counts()
#dummy

In [ ]:
data=train_df.groupby(["trafficSource.adContent"])["totals.totalTransactionRevenue"].sum().sort_values(ascending=False)
data[1:10]

In [ ]:
train_df['trafficSource.adwordsClickInfo.gclId'].unique()

### Drop unnecessary columns

### Feature Engineering

In [ ]:
df_clean=train_df.drop(['visitId','geoNetwork.region','geoNetwork.metro','geoNetwork.city','domain',
                        'totals.transactions','totals.transactionRevenue','trafficSource.adwordsClickInfo.isVideoAd',
                        'trafficSource.source','trafficSource.campaign','trafficSource.referralPath','visitNumber',
                        'visitStartTime','trafficSource.keyword','trafficSource.adwordsClickInfo.gclId','_day','_year',
                       'trafficSource.adContent'],axis=1)

In [ ]:
df_clean.head()

### Fill NaN values

In [ ]:
df_clean['totals.timeOnSite'] = train_df['totals.timeOnSite'].fillna(0)
df_clean['totals.totalTransactionRevenue'] = train_df['totals.totalTransactionRevenue'].fillna(0.0)
df_clean['totals.pageviews']=df_clean['totals.pageviews'].fillna(0)
df_clean['totals.sessionQualityDim']=df_clean['totals.sessionQualityDim'].fillna(0)
df_clean['totals.sessionQualityDim']=df_clean['totals.sessionQualityDim'].astype('category')
df_clean['_weekday']=df_clean['_weekday'].astype('category')
df_clean['_month']=df_clean['_month'].astype('category')
df_clean['_visitHour']=df_clean['_visitHour'].astype('category')

In [ ]:
df_clean.info()

In [ ]:
df_clean['month_4']=0
df_clean.loc[df_clean['_month']==4,'month_4']=1
df_clean['isTuesdays']=0
df_clean.loc[df_clean['_weekday']==2,'isTuesdays']=1
df_clean['BS_Firefox']=0
df_clean.loc[df_clean['device.browser']=='Firefox','BS_Firefox']=1
df_clean['BS_Chrome']=0
df_clean.loc[df_clean['device.browser']=='Chrome','BS_Chrome']=1
df_clean['BS_Safari']=0
df_clean.loc[df_clean['device.browser']=='Safari','BS_Safari']=1
df_clean['BS_IE']=0
df_clean.loc[df_clean['device.browser']=='Internet Explorer','BS_IE']=1
df_clean['BS_Android']=0
df_clean.loc[df_clean['device.browser']=='Android Webview','BS_Android']=1


df_clean['OS_Windows']=0
df_clean.loc[df_clean['device.operatingSystem']=='Windows','OS_Windows']=1
df_clean['OS_Macintosh']=0
df_clean.loc[df_clean['device.operatingSystem']=='Macintosh','OS_Macintosh']=1
df_clean['OS_Android']=0
df_clean.loc[df_clean['device.operatingSystem']=='Android','OS_Android']=1
df_clean['OS_iOS']=0
df_clean.loc[df_clean['device.operatingSystem']=='iOS','OS_iOS']=1


#subContinent1: Northern America (highest non-zero revenue)
df_clean['subCont_NorthernAmerica']=0
df_clean.loc[df_clean['geoNetwork.subContinent']=='Northern America','subCont_NorthernAmerica']=1

#subContinent2: Western Africa (highest mean revenue)
df_clean['subCont_Western Africa']=0
df_clean.loc[df_clean['geoNetwork.subContinent']=='Western Africa','subCont_Western Africa']=1
#country1: US (highest non-zero revenue)
df_clean['country_USA']=0
df_clean.loc[df_clean['geoNetwork.country']=='United States','country_USA']=1
#country2: Australia (highest mean revenue)
df_clean['country_Australia']=0
df_clean.loc[df_clean['geoNetwork.country']=='Australia','country_Australia']=1

#traffic medium
df_clean['medium_cpm']=0
df_clean.loc[df_clean['trafficSource.medium']=='cpm','medium_cpm']=1

In [ ]:
#get dummies
df_clean=df_clean.join(pd.get_dummies(df_clean['channelGrouping'])).drop(['(Other)'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['_visitHour'],prefix='Hour')).drop(['Hour_1','Hour_4','Hour_5','Hour_6','Hour_7','Hour_16','Hour_17','Hour_18','Hour_19','Hour_20','Hour_21','Hour_22','Hour_23'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['device.isMobile'],prefix='Mobile',drop_first=True))
df_clean=df_clean.join(pd.get_dummies(df_clean['device.deviceCategory'])).drop(['tablet','mobile'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['geoNetwork.continent'])).drop(['(not set)'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['trafficSource.isTrueDirect'],prefix='isTrueDirect',drop_first=True))

In [ ]:
df_clean['sessionQuality.100'] = pd.get_dummies(df_clean['totals.sessionQualityDim'])[100]
df_clean['Slot.RHS'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.slot'])['RHS']
df_clean['Slot.Top'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.slot'])['Top']
df_clean['Network.Content'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Content']
df_clean['Network.GSearch'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Google Search']
df_clean['Network.PSearch'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Search partners']

In [ ]:
df_clean=df_clean.drop(['_month','channelGrouping','_weekday','device.operatingSystem','geoNetwork.country',
                       'geoNetwork.subContinent','geoNetwork.continent','geoNetwork.networkDomain','device.isMobile',
                       'device.deviceCategory','device.browser','trafficSource.medium','trafficSource.isTrueDirect',
                       '_visitHour','trafficSource.adwordsClickInfo.adNetworkType',
              'trafficSource.adwordsClickInfo.slot',
              'totals.sessionQualityDim'],axis=1)

In [ ]:
df_clean.head()

In [ ]:
df_clean.info()

### Produce Transaction Level of Dataset for Modeling

In [ ]:
df_clean = df_clean.rename(columns={'totals.totalTransactionRevenue': 'revenue', 'fullVisitorId': 'id',
                                    'trafficSource.adwordsClickInfo.page': 'TS_adwordsClickInfo.page',
                                    'Affiliates': 'CG_Affiliates', 'Direct': 'CG_Direct','Display':'CG_Display',
                                    'Organic Search': 'CG_organicSearch',
                                   'Paid Search': 'CG_paidSearch' ,'Referral': 'CG_Referral',
                                   'Social': 'CG_Social', 'Mobile_True': 'device_Mobile',
                                   'desktop': 'device_Desktop', 'Africa': 'cont_Africa',
                                   'Americas': 'cont_Americas', 'Asia': 'cont_Asia',
                                   'Europe': 'cont_Europe', 'Oceania': 'cont_Oceania',
                                   'isTrueDirect_False': 'TS_isTrueDirect', 'sessionQuality.100': 'TS_sessionQuality.100',
                                   'Slot.RHS': 'TS_Slot.RHS', 'Slot.Top': 'TS_Slot.Top',
                                    'Network.Content': 'TS_Network.Content', 'Network.GSearch': 'TS_Network.GSearch',
                                    'Network.PSearch': 'TS_Network.PSearch'})

In [ ]:
df_clean.info()

In [ ]:
df_clean.to_pickle("trasaction_dataset.pkl")

### Produce Aggregation to User level of Dataset for Modeling

In [ ]:
#customer dataframe cf
cf1 = pd.DataFrame()  #float data
cf2 = pd.DataFrame()  #dummy data
cf3 = pd.DataFrame()  #target revenue

In [ ]:
cf1 = df_clean.iloc[:,0:6]
cf1 = cf1.groupby(['id']).agg(['sum','mean'])
cf1.columns = ['_'.join(col).strip() for col in cf1.columns.values]
#cf1.head()

In [ ]:
select_columns = [1]
select_columns.extend(range(8,57))
cf2 = df_clean[df_clean.columns[select_columns]]
cf2 = cf2.groupby(['id']).agg(['max'])
cf2.columns = ['_'.join(col).strip() for col in cf2.columns.values]
cf2.head()

In [ ]:
cf3 = df_clean.groupby(['id'])['revenue'].agg(['sum'])
cf3['logRevenue'] = np.log(1+cf3['sum'])
cf3 = cf3.drop('sum', axis = 1)

In [ ]:
# comebine customer dataset
cf = pd.concat([cf3, cf1, cf2], axis=1)
cf = cf.reset_index()
cf = pd.DataFrame(cf)
cf.shape

In [ ]:
cf.head()

In [ ]:
# save to pickle
cf.to_pickle("customer_dataset.pkl")

In [ ]:
cf = pd.read_pickle('customer_dataset.pkl')

In [ ]:
# drop time columns
time_columns = ['month_4_max','isTuesdays_max','Hour_0_max','Hour_2_max','Hour_3_max','Hour_8_max','Hour_9_max','Hour_10_max','Hour_11_max','Hour_12_max','Hour_13_max','Hour_14_max','Hour_15_max']

In [ ]:
cf = cf.drop(time_columns, axis=1)

In [ ]:
cf.info()

### Graph Analysis

In [ ]:
# import warnings
import networkx as nx
# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")
# read the data
Model_df = pd.read_pickle('trasaction_dataset.pkl')

In [ ]:
Model_df.head()

In [ ]:
Model_df.shape

## Aggregated

In [ ]:
##Aggregating
ID_df = Model_df.groupby(['id']).agg({'revenue':'sum'}).reset_index()
ID_df['logRev'] = np.log1p(ID_df['revenue'])
ID_df['logRev_t'] =round(ID_df['logRev'], -1)
ID_df = ID_df.drop(['revenue', 'logRev'],axis = 1)
ID_df1 = ID_df
ID_df1['id_1'] = ID_df1['id']
ID_df1 = ID_df1.drop('id',axis = 1)
ID_df = ID_df.drop('id_1',axis = 1)

In [ ]:
ID_df.head()

In [ ]:
ID_df1.head()

In [ ]:
ID_df1.shape

In [ ]:
joinme = pd.merge(ID_df1.head(100),
                  ID_df.head(100), on='logRev_t', how='inner')
joinme = joinme[joinme['id'] != joinme['id_1']]
#joinme = ID_df.join(ID_df1, lsuffix='_1', rsuffix='_2')

In [ ]:
#greater = joinme[joinme['logRev_t' ] > 0 ]

In [ ]:
#greater.head()
joinme['logRev_t'].describe()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

FG = nx.from_pandas_edgelist(joinme, source='id',
                             target='id_1', edge_attr=True)

In [ ]:
plt.figure(figsize=(10,8))
nx.draw(FG, with_labels=False)
plt.title()
plt.show()

## Transaction

In [ ]:
## Not Aggregating
Trans_df = Model_df[['id', 'revenue']]
    # Getting rid of zeros
Trans_df = Trans_df[Trans_df['revenue'] > 0]
    # Smaller Values
Trans_df['revenue'] = np.log1p(Trans_df['revenue'] )
Trans_df['revenue'] =round(Trans_df['revenue'], -1)

Trans_df.head()

In [ ]:
Trans_df.shape

In [ ]:
joinme = pd.merge(Trans_df.head(1000),
                  Trans_df.head(1000), on='revenue', how='inner')

In [ ]:
joinme = joinme[joinme['id_x'] != joinme['id_y']]
joinme.head()

In [ ]:
joinme.shape

In [ ]:
TG = nx.from_pandas_edgelist(joinme, source='id_x',
                             target='id_y', edge_attr=True)

In [ ]:
plt.figure(figsize=(10,8))
nx.draw(TG, with_labels=False)
plt.title('Transaction Revenue Graph')
plt.show()

# Country Graph Analysis

In [ ]:
# read the data
Data_df = pd.read_pickle('2017_clean.pkl')

In [ ]:
Data_df.head()

In [ ]:
Data_df.shape

In [ ]:
country_df = Data_df.loc[:, ['fullVisitorId', 'totals.transactionRevenue', 'geoNetwork.country']]
country_df
country_df['totals.transactionRevenue'] = country_df['totals.transactionRevenue'].fillna(0)
country_df.head()


In [ ]:
this = country_df.loc[:,['fullVisitorId', 'geoNetwork.country']].groupby(['fullVisitorId','geoNetwork.country']).count()
thisone = pd.DataFrame(data=this)
thisone = thisone.reset_index().groupby(['fullVisitorId'])['geoNetwork.country'].count()
thisone = pd.DataFrame(data=thisone)
thisone.head()
#newone.head()
newone = thisone[thisone['geoNetwork.country'] > 1]
#this.head()['fullVisitorId'].groupby(['fullVisitorId']).count()
#this.fullVisitorId.value_counts().reset_index(name="count").query("count > 1")

In [ ]:
newone.shape
newone.head()

In [ ]:
newone = newone.reset_index()

In [ ]:
newone['fullVisitorId'].values

### Comparing Revenue

In [ ]:
joinme = pd.merge(newone['fullVisitorId'],
                  country_df, on='fullVisitorId', how='left')
joinme.head()

In [ ]:
joinme.shape

In [ ]:
jjiin = pd.merge(joinme.loc[:,['fullVisitorId', 'totals.transactionRevenue']],
                 joinme.loc[:,['fullVisitorId', 'totals.transactionRevenue']],
                 on = 'totals.transactionRevenue', how = 'inner')
jjiin.head()

In [ ]:
jjiin.shape

In [ ]:
TG = nx.from_pandas_edgelist(jjiin, source='fullVisitorId_x',
                             target='fullVisitorId_y', edge_attr=True)

In [ ]:
plt.figure(figsize=(20,20))
nx.draw(TG, with_labels=True)
plt.title('Multiple Countries: Revenue')
plt.show()

In [ ]:
Data_df[Data_df['fullVisitorId'] == '7745913892709272663']

In [ ]:
joinme[joinme['fullVisitorId'] == '7745913892709272663']

### Comparing Country

In [ ]:
jjiin = pd.merge(joinme.loc[:,['fullVisitorId', 'geoNetwork.country']],
                 joinme.loc[:,['fullVisitorId', 'geoNetwork.country']],
                 on = 'geoNetwork.country', how = 'inner')
jjiin.head()

In [ ]:
CG = nx.from_pandas_edgelist(jjiin, source='fullVisitorId_x',
                             target='fullVisitorId_y', edge_attr=True)

In [ ]:
plt.figure(figsize=(20,20))
nx.draw(CG, with_labels=True)
plt.title('Multiple Countries: Country')
plt.show()

## Modeling

### Modeling on Transaction Level

In [ ]:
# read the data
Model_df = pd.read_pickle('trasaction_dataset.pkl')

In [ ]:
# Importing Libraries
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve,GroupShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
# Importing Classifier Modules
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as MSE
import sklearn.model_selection as cv

In [ ]:
#split train test datset
#Split dataset with individual id instead of spliting it into transaction levels
seed=3
X = Model_df.drop(['revenue'],axis = 1)
y = Model_df.loc[:,['id','revenue']]
#np.log1p()
train_inds,test_inds= next(GroupShuffleSplit(test_size=0.20,n_splits=2,random_state=3).split(X,groups=X['id']))
X_train=X.loc[train_inds]
X_test =X.loc[test_inds]
y_train=y.loc[train_inds]
y_test=y.loc[test_inds]


y_train=np.log1p(y_train.drop(['id'],axis=1)).values[:,0]
y_test=np.log1p(y_test.drop(['id'],axis=1)).values[:,0]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20,random_state=seed)

In [ ]:
train_date=X_train['date']
train_id=X_train['id']
X_train=X_train.drop(['id','date'],axis=1)
test_date=X_test['date']
test_id=X_test['id']
X_test=X_test.drop(['id','date'],axis=1)

Target, prediction process
* 1st log1p to target
* 2nd exmp1 predictions
* 3rd sum predictions
* 4th log1p to sum

**Linear Model**

In [ ]:
reg = LinearRegression()
reg.fit(X_train,y_train)

**RMSE results for Models**

In [ ]:
predictions = reg.predict(X_test)
mse_dt = MSE(predictions, y_test)
# Compute rmse_dt
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
# Import mean_squared_error from sklearn.metrics as MSE
from sklearn.metrics import mean_squared_error as MSE

# Predict the labels of the training set
y_pred_train = reg.predict(X_train)

# Evaluate the training set RMSE of dt
RMSE_train = (MSE(y_train, y_pred_train))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
scores = cross_val_score(reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
score_description = " %0.5f (+/- %0.5f)" % (np.sqrt(scores.mean()*-1), scores.std() * 2)

In [ ]:
print('{model:25} CV-5 RMSE: {score}'.format(
 model=reg.__class__.__name__,
 score=score_description
 ))

**Visual Presentation of Model results**

In [ ]:
log_revenue_compare=pd.DataFrame({'date': train_date, 'log_revenue_pred': y_pred_train,'log_revenue_actual':y_train})

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.lineplot(x="date", y="log_revenue_pred", data=log_revenue_compare,color='b')
ax = sns.lineplot(x="date", y="log_revenue_actual", data=log_revenue_compare,color='r')

**RMSE after aggregating into user level**

In [ ]:
y_pred_train[y_pred_train<0]=0

In [ ]:
revenue_per_user=pd.DataFrame({'user_id': np.array(train_id), 'revenue_pred': np.expm1(y_pred_train),'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
predictions[predictions<0]=0

In [ ]:
revenue_per_user_test=pd.DataFrame({'user_id': test_id, 'revenue_pred': np.expm1(predictions), 'revenue_actual': np.expm1(y_test)})
revenue_per_user_test=revenue_per_user_test.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user_test['log_sum_pred']=np.log1p(revenue_per_user_test['revenue_pred'])
revenue_per_user_test['log_sum_actual']=np.log1p(revenue_per_user_test['revenue_actual'])
revenue_per_user_test=revenue_per_user_test.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
mse_dt = (MSE(revenue_per_user_test['log_sum_pred'], revenue_per_user_test['log_sum_actual']))**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
###Cross validation for aggregate results
from sklearn.model_selection import cross_val_predict
# Compute the array containing the 10-folds CV MSEs


cross_val_pred = cross_val_predict(reg, X_train, y_train, cv=10,
                                  groups = train_id,
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
#RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))


cross_val_pred[cross_val_pred<0] = 0


revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(cross_val_pred),
                               'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum',
                                                            'revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)


################NEW BOX#################################################

###TESTING NEW WAY
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('CV RMSE: {}'.format(RMSE_train))

**XGBoost**

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.05, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=5)



xgb.fit(X_train,y_train)
predictions = xgb.predict(X_test)

**RMSE results for Models**

In [ ]:
mse_dt = MSE(predictions, y_test)
# Compute rmse_dt
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
# Import mean_squared_error from sklearn.metrics as MSE
from sklearn.metrics import mean_squared_error as MSE

# Predict the labels of the training set
y_pred_train = xgb.predict(X_train)

# Evaluate the training set RMSE of dt
RMSE_train = (MSE(y_train, y_pred_train))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
from sklearn.model_selection import cross_val_score
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(xgb, X_train, y_train, cv=10,
                                  scoring='neg_mean_squared_error',
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))

In [ ]:
print('CV RMSE: {}'.format(RMSE_CV))

**RMSE after aggregating into user level**

In [ ]:
y_pred_train[y_pred_train<0]=0

In [ ]:
revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(y_pred_train), 'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
predictions[predictions<0]=0

In [ ]:
revenue_per_user_test=pd.DataFrame({'user_id': test_id, 'revenue_pred': np.expm1(predictions), 'revenue_actual': np.expm1(y_test)})
revenue_per_user_test=revenue_per_user_test.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user_test['log_sum_pred']=np.log1p(revenue_per_user_test['revenue_pred'])
revenue_per_user_test['log_sum_actual']=np.log1p(revenue_per_user_test['revenue_actual'])
revenue_per_user_test=revenue_per_user_test.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
mse_dt = (MSE(revenue_per_user_test['log_sum_pred'], revenue_per_user_test['log_sum_actual']))**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
###Cross validation for aggregate results
from sklearn.model_selection import cross_val_predict
# Compute the array containing the 10-folds CV MSEs


cross_val_pred = cross_val_predict(xgb, X_train, y_train, cv=10,
                                  groups = train_id,
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
#RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))


cross_val_pred[cross_val_pred<0] = 0


revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(cross_val_pred),
                               'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum',
                                                            'revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)


################NEW BOX#################################################

###TESTING NEW WAY
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('CV RMSE: {}'.format(RMSE_train))

**Gradiant Boost**

In [ ]:
from sklearn import ensemble
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 5, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gradient = ensemble.GradientBoostingRegressor(**params)

gradient.fit(X_train, y_train)

**RMSE results for Models**

In [ ]:
# Predict the test set labels 'y_pred'
predictions = gradient.predict(X_test)
y_pred_train=gradient.predict(X_train)

# Evaluate the test set RMSE
rmse_test = MSE(y_test, predictions)**(1/2)
rmse_train = MSE(y_train, y_pred_train)**(1/2)

print("Test set RMSE of dt: {}".format(rmse_dt))
print('Train RMSE: {}'.format(rmse_train))

In [ ]:
from sklearn.model_selection import cross_val_score
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(gradient, X_train, y_train, cv=10,
                                  scoring='neg_mean_squared_error',
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
print('CV RMSE: {}'.format(RMSE_CV))

**RMSE after aggregating into user level**

In [ ]:
y_pred_train[y_pred_train<0]=0

In [ ]:
revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(y_pred_train), 'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
predictions[predictions<0]=0

In [ ]:
revenue_per_user_test=pd.DataFrame({'user_id': test_id, 'revenue_pred': np.expm1(predictions), 'revenue_actual': np.expm1(y_test)})
revenue_per_user_test=revenue_per_user_test.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user_test['log_sum_pred']=np.log1p(revenue_per_user_test['revenue_pred'])
revenue_per_user_test['log_sum_actual']=np.log1p(revenue_per_user_test['revenue_actual'])
revenue_per_user_test=revenue_per_user_test.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
mse_dt = (MSE(revenue_per_user_test['log_sum_pred'], revenue_per_user_test['log_sum_actual']))**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
###Cross validation for aggregate results
from sklearn.model_selection import cross_val_predict
# Compute the array containing the 10-folds CV MSEs


cross_val_pred = cross_val_predict(gradient, X_train, y_train, cv=10,
                                  groups = train_id,
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
#RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))


cross_val_pred[cross_val_pred<0] = 0


revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(cross_val_pred),
                               'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum',
                                                            'revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)


################NEW BOX#################################################

###TESTING NEW WAY
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('CV RMSE: {}'.format(RMSE_train))

In [ ]:
# Create a pd.Series of features importances
importances_grad = pd.Series(gradient.feature_importances_,
index = X_train.columns)

# Sort importances_rf
sorted_importances_grad = importances_grad.sort_values()

# Make a horizontal bar plot
plt.figure(figsize=(12, 10))
sorted_importances_grad.plot(kind='barh', color='blue')
plt.show()

**Random Forest**

In [ ]:
rfr=RandomForestRegressor(max_depth=6, random_state=seed,n_estimators=500)
rfr.fit(X_train,y_train)

**RMSE results for Models**

In [ ]:
predictions=rfr.predict(X_test)
mse_dt = MSE(predictions, y_test)
# Compute rmse_dt
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
# Import mean_squared_error from sklearn.metrics as MSE
from sklearn.metrics import mean_squared_error as MSE

# Predict the labels of the training set
y_pred_train = rfr.predict(X_train)

# Evaluate the training set RMSE of dt
RMSE_train = (MSE(y_train, y_pred_train))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

**RMSE after aggregating into user level**

In [ ]:
y_pred_train[y_pred_train<0]=0

In [ ]:
revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(y_pred_train), 'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
predictions[predictions<0]=0

In [ ]:
revenue_per_user_test=pd.DataFrame({'user_id': test_id, 'revenue_pred': np.expm1(predictions), 'revenue_actual': np.expm1(y_test)})
revenue_per_user_test=revenue_per_user_test.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user_test['log_sum_pred']=np.log1p(revenue_per_user_test['revenue_pred'])
revenue_per_user_test['log_sum_actual']=np.log1p(revenue_per_user_test['revenue_actual'])
revenue_per_user_test=revenue_per_user_test.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
mse_dt = (MSE(revenue_per_user_test['log_sum_pred'], revenue_per_user_test['log_sum_actual']))**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
###Cross validation for aggregate results
from sklearn.model_selection import cross_val_predict
# Compute the array containing the 10-folds CV MSEs


cross_val_pred = cross_val_predict(rfr, X_train, y_train, cv=10,
                                  groups = train_id,
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
#RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))


cross_val_pred[cross_val_pred<0] = 0


revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(cross_val_pred),
                               'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum',
                                                            'revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)


################NEW BOX#################################################

###TESTING NEW WAY
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('CV RMSE: {}'.format(RMSE_train))

**Decision Tree**

In [ ]:
tree = DecisionTreeRegressor()

In [ ]:
# Instantiate

# Fit dt to the training set
tree.fit(X_train, y_train)

# Compute y_pred
y_pred = tree.predict(X_test)

# Evaluate the test set RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)
rmse_train = MSE(y_train, y_pred_train)**(1/2)

In [ ]:
print(rmse_test)
print(rmse_train)

In [ ]:
from sklearn.model_selection import cross_val_score
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(tree, X_train, y_train, cv=5,
                                  scoring='neg_mean_squared_error',
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
print('CV RMSE: {}'.format(RMSE_CV))

In [ ]:
# Create a pd.Series of features importances
importances_tree = pd.Series(tree.feature_importances_,
index = X_train.columns)

# Sort importances_rf
sorted_importances_tree = importances_tree.sort_values()

# Make a horizontal bar plot
plt.figure(figsize=(12, 8))
sorted_importances_tree.plot(kind='barh', color='blue')
plt.show()

**LightGBM**

In [ ]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(random_state=0)
regressors = [('LightGBM', lgb)]


In [ ]:
for clf_name, clf in regressors:
    clf.fit(X_train, y_train.ravel())
    predictions = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    RMSE_train = (MSE(y_train, y_pred_train))**(1/2)
    print('{} Train RMSE: {:.4f}'.format(clf_name, RMSE_train))
    y_pred_test = clf.predict(X_test)
    RMSE_test = (MSE(y_test, y_pred_test))**(1/2)
    print("{} Test RMSE: {:.4f}".format(clf_name, RMSE_test))

In [ ]:
# cross validation
for clf_name, clf in regressors:
    MSE_CV_scores = - cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    RMSE_CV = (MSE_CV_scores.mean())**(1/2)
    print('{} CV RMSE: {:.4f}'.format(clf_name, RMSE_CV))

**RMSE after aggregating into user level**

In [ ]:
y_pred_train[y_pred_train<0]=0

In [ ]:
revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(y_pred_train), 'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('Train RMSE: {}'.format(RMSE_train))

In [ ]:
predictions[predictions<0]=0

In [ ]:
revenue_per_user_test=pd.DataFrame({'user_id': test_id, 'revenue_pred': np.expm1(predictions), 'revenue_actual': np.expm1(y_test)})
revenue_per_user_test=revenue_per_user_test.groupby(['user_id']).agg({'revenue_pred':'sum','revenue_actual':'sum'}).reset_index()
revenue_per_user_test['log_sum_pred']=np.log1p(revenue_per_user_test['revenue_pred'])
revenue_per_user_test['log_sum_actual']=np.log1p(revenue_per_user_test['revenue_actual'])
revenue_per_user_test=revenue_per_user_test.drop(['revenue_pred','revenue_actual'],axis=1)

In [ ]:
rmse_dt = (MSE(revenue_per_user_test['log_sum_pred'], revenue_per_user_test['log_sum_actual']))**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {}".format(rmse_dt))

In [ ]:
###Cross validation for aggregate results
from sklearn.model_selection import cross_val_predict
# Compute the array containing the 10-folds CV MSEs


cross_val_pred = cross_val_predict(lgb, X_train, y_train, cv=10,
                                  groups = train_id,
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
#RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
#print('CV RMSE: {}'.format(RMSE_CV))


cross_val_pred[cross_val_pred<0] = 0


revenue_per_user=pd.DataFrame({'user_id': train_id, 'revenue_pred': np.expm1(cross_val_pred),
                               'revenue_actual': np.expm1(y_train)})
revenue_per_user=revenue_per_user.groupby(['user_id']).agg({'revenue_pred':'sum',
                                                            'revenue_actual':'sum'}).reset_index()
revenue_per_user['log_sum_pred']=np.log1p(revenue_per_user['revenue_pred'])
revenue_per_user['log_sum_actual']=np.log1p(revenue_per_user['revenue_actual'])
revenue_per_user=revenue_per_user.drop(['revenue_pred','revenue_actual'],axis=1)


################NEW BOX#################################################

###TESTING NEW WAY
# Evaluate the training set RMSE of dt
RMSE_train = (MSE(revenue_per_user['log_sum_pred'], revenue_per_user['log_sum_actual']))**(1/2)

# Print RMSE_train
print('CV RMSE: {}'.format(RMSE_train))

### Modeling on user Level

In [ ]:
# load required library
%matplotlib inline
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Importing Classifier Modules
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# cross validation
import sklearn
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load data
df = pd.read_pickle('customer_dataset.pkl')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
X = df.drop(['id','logRevenue'], axis=1).values
y = df['logRevenue'].values

# Split data into 75% train and 25% test
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
gb = GradientBoostingRegressor()
kn = KNeighborsRegressor()
rf = RandomForestRegressor()

In [ ]:
regressors = [('Linear Regression', lr),
              ('Decision Tree Regressor', dt),
              ('Gradient Boosting Regressor', gb),
              ('KNeighbors Regressor', kn),
              ('Random Forest Regressor', rf)
             ]

In [ ]:
for clf_name, clf in regressors:
    clf.fit(X_train, y_train.ravel())
    predicted = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    RMSE_train = (MSE(y_train, y_pred_train))**(1/2)
    print('{} Train RMSE: {:.4f}'.format(clf_name, RMSE_train))
    y_pred_test = clf.predict(X_test)
    RMSE_test = (MSE(y_test, y_pred_test))**(1/2)
    print("{} Test RMSE: {:.4f}".format(clf_name, RMSE_test))

In [ ]:
# cross validation
for clf_name, clf in regressors:
    MSE_CV_scores = - cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    RMSE_CV = (MSE_CV_scores.mean())**(1/2)
    print('{} CV RMSE: {:.4f}'.format(clf_name, RMSE_CV))

### Light GBM

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgb = LGBMRegressor(random_state=0)

In [ ]:
regressors = [('LightGBM', lgb)]

In [ ]:
for clf_name, clf in regressors:
    clf.fit(X_train, y_train.ravel())
    predicted = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    RMSE_train = (MSE(y_train, y_pred_train))**(1/2)
    print('{} Train RMSE: {:.4f}'.format(clf_name, RMSE_train))
    y_pred_test = clf.predict(X_test)
    RMSE_test = (MSE(y_test, y_pred_test))**(1/2)
    print("{} Test RMSE: {:.4f}".format(clf_name, RMSE_test))

In [ ]:
# cross validation
for clf_name, clf in regressors:
    MSE_CV_scores = - cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    RMSE_CV = (MSE_CV_scores.mean())**(1/2)
    print('{} CV RMSE: {:.4f}'.format(clf_name, RMSE_CV))

In [ ]:
# Tune Paramaters with lightgbm

# Create parameters to search
gridParams = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2], # default 0.1
    'n_estimators': [40, 100, 150], # default 100
    'num_leaves': [21,31,41], # default 31
    'random_state': [0], # Updated from 'seed'
    'max_depth': [-1, 20],
    }

In [ ]:
lgb_grid = LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression',
          n_jobs = 3, # Updated from 'nthread'
          max_depth = 1000)

# Create the grid
grid = GridSearchCV(lgb_grid, gridParams, cv=5)
grid.fit(X_train, y_train)

In [ ]:
print('Best parameters found by grid search are:', grid.best_params_)

In [ ]:
y_pred_train = grid.predict(X_train)
RMSE_train = (MSE(y_train, y_pred_train))**(1/2)
print('LightGBM Tuned Train RMSE: {:.4f}'.format(RMSE_train))
y_pred_test = grid.predict(X_test)
RMSE_test = (MSE(y_test, y_pred_test))**(1/2)
print("LightGBM Tuned RMSE: {:.4f}".format(RMSE_test))

In [ ]:
# Tuned Set cross validation
lgbm_tuned = LGBMRegressor(learning_rate= 0.05, max_depth= -1, n_estimators=150, num_leaves= 31, random_state=0)
regressors = [('LightGBM Tuned', lgbm_tuned)]
for clf_name, clf in regressors:
    MSE_CV_scores = - cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    RMSE_CV = (MSE_CV_scores.mean())**(1/2)
    print('{} CV RMSE: {:.4f}'.format(clf_name, RMSE_CV))

### Ridge Regression Model

In [ ]:
ridge = RidgeCV(cv=10)

# fit or train the linear regression model on the training set and store␣
ridge.fit(X_train, y_train)
# show the alpha parameter used in final ridgeCV model ridge.alpha_
ridge.alpha_

#
train_pred = ridge.predict(X_train)
ridge_test_pred = ridge.predict(X_test)

In [ ]:
#print RMSE of training predictions
print('RMSE on train for regression: ', np.sqrt(MSE(y_train, train_pred)))
print('RMSE on test for regression: ', np.sqrt(MSE(y_test,ridge_test_pred)))

In [ ]:
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(ridge, X_train, y_train, cv=10,
                                  scoring='neg_mean_squared_error',
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
print('CV RMSE: {}'.format(RMSE_CV))


### Gradient Model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gradient = ensemble.GradientBoostingRegressor(**params)

gradient.fit(X_train, y_train)

In [ ]:
# Predict the test set labels 'y_pred'
y_pred = gradient.predict(X_test)
y_pred_train=gradient.predict(X_train)

# Evaluate the test set RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)
rmse_train = MSE(y_train, y_pred_train)**(1/2)

In [ ]:
# Create a pd.Series of features importances
importances_grad = pd.Series(gradient.feature_importances_,
index = X_train.columns)

# Sort importances_rf
sorted_importances_grad = importances_grad.sort_values()

# Make a horizontal bar plot
plt.figure(figsize=(12, 8))
sorted_importances_grad.plot(kind='barh', color='blue')
plt.show()

In [ ]:
# Print the test set RMSE
print('Train set RMSE for gradient on train data: {:.3f}'.format(rmse_train))
print('Test set RMSE for gradient on test data: {:.3f}'.format(rmse_test))

In [ ]:
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(gradient, X_train, y_train, cv=10,
                                  scoring='neg_mean_squared_error',
                                  n_jobs=-1)

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
print('CV RMSE: {}'.format(RMSE_CV))


### XGBoost Model

In [ ]:
# xgb models only accept 'DMatrix' input; convert the data here
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
# parameter grid for xgb model
param_dict = { 'max_depth':6, 'min_child_weight': 1,
              'eta':.1, 'subsample': 1,
              'colsample_bytree': 1, # Other parameters 'objective':'reg:linear', 'eval_metric':'rmse'
}
# train XGB model on split training data using split test data
num_boost_round=999
model_xgb = xgb.train( param_dict, dtrain,
                  num_boost_round=num_boost_round,
                  evals=[(dtest, "Test")],
                  early_stopping_rounds=10)

In [ ]:
dpred=xgb.DMatrix(X_train)

# predict RMSE of model on training data
print("RMSE of XGB model on train data: ", np.sqrt(MSE(model_xgb.predict(dpred), y_train)))
# predict RMSE of model on test data
model_xgb_test_pred = model_xgb.predict(xgb.DMatrix(X_test))
print("RMSE of XGB model on test data: ", np.sqrt(MSE(model_xgb_test_pred, y_test)))

In [ ]:
plt.figure(figsize=(20, 20))
plt.show(plot_importance(model_xgb))

## Customer Segmentation and LTV Prediction

In this section, we try to do customer segmentation with RFM (Recency, Frequency and Monetary Value) scores. Since we have the data for the whole year of 2017, we seperate it into two sets: January-September and Octobor-December. Then we will do customer segmentation for the first 9 months and predict the customer life time value (LTV) for the last 3 months.

**Customer Segmentation (First 9 Months)** <br>
We calculate the recency (how many days past since the last transaction), frequency (how many transactions within 9 months) and log revenue for all transactions. Then we do clustering for these 3 features so that we have 3 clustering features. We re-order the cluster order the make the biggest cluster number represents the best results. Thus, we calculate the overall score by summing up these 3 scores. Then we segment the customer into 4 groups according to the overall scores.

**LTV Prediction (Last 3 Months)**  <br>
First, we use the log revenue of the last 3 months to do a LTV clustering. Then we use the 9-month features to predict the 3-month LTV.

In [ ]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pyoff.init_notebook_mode()

In [ ]:
# load data
df = pd.read_pickle('2017_clean.pkl')
train = df.copy()

In [ ]:
# keep related information and change data type
train = train[['fullVisitorId', 'date', 'totals.totalTransactionRevenue']]
train["date"] = pd.to_datetime(train["date"], format="%Y%m%d") # seting the column as pandas datetime
train['totals.totalTransactionRevenue']=train['totals.totalTransactionRevenue'].astype('float')

In [ ]:
train.head()

In [ ]:
train['date'].describe()

In [ ]:
# select data from Jan to Sept
train_9 = train[(train.date < date(2017,10,1)) & (train.date >= date(2017,1,1))].reset_index(drop=True)
# select data from Oct to Dec
train_12 = train[(train.date >= date(2017,10,1)) & (train.date <= date(2017,12,31))].reset_index(drop=True)

In [ ]:
train_9['date'].describe()

In [ ]:
train_12['date'].describe()

In [ ]:
customer = pd.DataFrame(train['fullVisitorId'].unique())
customer.columns = ['fullVisitorId']

### Recency

In [ ]:
max_purchase = train_9.groupby('fullVisitorId').date.max().reset_index()

In [ ]:
max_purchase.columns = ['fullVisitorId','MaxPurchaseDate']

In [ ]:
max_purchase['Recency'] = (max_purchase['MaxPurchaseDate'].max() - max_purchase['MaxPurchaseDate']).dt.days

In [ ]:
user_recency = pd.merge(customer, max_purchase[['fullVisitorId','Recency']], on='fullVisitorId')

In [ ]:
user_recency.head()

In [ ]:
user_recency.Recency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=user_recency['Recency']
    )
]

plot_layout = go.Layout(
        title='Recency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
# Elbow Method
sse={}
recency = user_recency[['Recency']]
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(recency)
    recency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# let's try 4 first
kmeans = KMeans(n_clusters=4)
kmeans.fit(user_recency[['Recency']])
user_recency['RecencyCluster'] = kmeans.predict(user_recency[['Recency']])

In [ ]:
user_recency.groupby('RecencyCluster')['Recency'].describe()

In [ ]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [ ]:
# call the funtion above to order cluster
user_recency = order_cluster('RecencyCluster', 'Recency', user_recency,False)

In [ ]:
user_recency.groupby('RecencyCluster')['Recency'].describe()

In [ ]:
user_recency.groupby('RecencyCluster')['Recency'].agg(['count','mean']).reset_index()

In [ ]:
user_recency.groupby('RecencyCluster')['Recency'].hist()
plt.xlabel('Recency')
plt.ylabel('Count')
plt.title('Recency_Cluster')

### Frequency

In [ ]:
user_frequency = train_9.groupby('fullVisitorId').date.count().reset_index()

In [ ]:
user_frequency.columns = ['fullVisitorId','Frequency']

In [ ]:
user_frequency.head()

In [ ]:
user = pd.merge(user_recency, user_frequency, on='fullVisitorId')

In [ ]:
user.head()

In [ ]:
user.Frequency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=user['Frequency']
    )
]

plot_layout = go.Layout(
        title='Frequency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
# Elbow Method
sse={}
frequency = user[['Frequency']]
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(frequency)
    frequency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(user[['Frequency']])
user['FrequencyCluster'] = kmeans.predict(user[['Frequency']])

In [ ]:
user = order_cluster('FrequencyCluster', 'Frequency',user,True)

In [ ]:
user.groupby('FrequencyCluster')['Frequency'].describe()

In [ ]:
user.groupby('FrequencyCluster')['Frequency'].agg(['count','mean']).reset_index()

### Monetary Value

In [ ]:
revenue = train_9.groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()

In [ ]:
revenue.columns = ['fullVisitorId', 'Revenue']
revenue['logRevenue'] = np.log(1+revenue['Revenue'])
revenue = revenue.drop(['Revenue'], axis=1)
revenue.head()

In [ ]:
user = pd.merge(user, revenue, on='fullVisitorId')
user.head()

In [ ]:
user.logRevenue.describe()

In [ ]:
# the elbow method show 2
kmeans = KMeans(n_clusters=2)
kmeans.fit(user[['logRevenue']])
user['logRevenueCluster'] = kmeans.predict(user[['logRevenue']])

In [ ]:
user = order_cluster('logRevenueCluster', 'logRevenue',user,True)

In [ ]:
user.groupby('logRevenueCluster')['logRevenue'].describe()

In [ ]:
user.groupby('logRevenueCluster')['logRevenue'].agg(['count','mean']).reset_index()

### Overall Segmentation

In [ ]:
user.head()

In [ ]:
user['OverallScore'] = user['RecencyCluster'] + user['FrequencyCluster'] + user['logRevenueCluster']

In [ ]:
user.groupby('OverallScore')['Recency','Frequency','logRevenue'].mean().reset_index()

Looks like overall score 7 is the highest value customer.

In [ ]:
user['Segment'] = 'Low-Value'
user.loc[user['OverallScore']>0,'Segment'] = 'Mid1-Value'
user.loc[user['OverallScore']>3,'Segment'] = 'Mid2-Value'
user.loc[user['OverallScore']>5,'Segment'] = 'High-Value'

In [ ]:
sg = user.groupby('Segment')['logRevenue'].agg(['count','mean']).sort_values(by=['mean']).reset_index()
sg.columns = ['Segment','count','mean_logRevenue_9']
sg

In [ ]:
user_sample = user.sample(20000)

In [ ]:
tx_graph = user_sample

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid1-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid1-Value'")['Frequency'],
        mode='markers',
        name='Mid1',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
            go.Scatter(
        x=tx_graph.query("Segment == 'Mid2-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid2-Value'")['Frequency'],
        mode='markers',
        name='Mid2',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Frequency"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = user_sample

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['logRevenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid1-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid1-Value'")['logRevenue'],
        mode='markers',
        name='Mid1',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
            go.Scatter(
        x=tx_graph.query("Segment == 'Mid2-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid2-Value'")['logRevenue'],
        mode='markers',
        name='Mid2',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['logRevenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "logRevenue"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = user_sample

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Low-Value'")['logRevenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid1-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Mid1-Value'")['logRevenue'],
        mode='markers',
        name='Mid1',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
            go.Scatter(
        x=tx_graph.query("Segment == 'Mid2-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Mid2-Value'")['logRevenue'],
        mode='markers',
        name='Mid2',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'High-Value'")['logRevenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "logRevenue"},
        xaxis= {'title': "Frequency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Life Time Value Prediction

In [ ]:
user.head()

In [ ]:
train_12.head()

In [ ]:
revenue_12 = train_12.groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
revenue_12.columns = ['fullVisitorId','Revenue_12']
revenue_12['logRevenue_12'] = np.log(1+revenue_12['Revenue_12'])
revenue_12 = revenue_12.drop(['Revenue_12'], axis=1)

In [ ]:
revenue_12.head()

In [ ]:
# merge 9-month features with 3-month revenue
merge = pd.merge(user, revenue_12, on='fullVisitorId', how='left')

In [ ]:
merge.head()

In [ ]:
merge = merge.fillna(0)

In [ ]:
sg_12 = merge.groupby('Segment')['logRevenue_12'].agg(['count','mean']).sort_values(by=['mean']).reset_index()
sg_12.columns = ['Segment','count','mean_logRevenue_12']
sg_12

In [ ]:
merge.shape

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(merge[['logRevenue_12']])
merge['LTVCluster'] = kmeans.predict(merge[['logRevenue_12']])

In [ ]:
merge = order_cluster('LTVCluster', 'logRevenue_12',merge,True)

In [ ]:
merge.groupby('LTVCluster')['logRevenue_12'].describe()

In [ ]:
ltv = merge.groupby('LTVCluster')['logRevenue_12'].agg(['count','mean']).reset_index()
ltv.columns = ['LTVCluster','count','mean_logRevenue_12']
ltv

In [ ]:
cluster = merge.copy()

In [ ]:
cluster.head()

In [ ]:
# dummy Segment
cluster['Segment_High']=0
cluster.loc[cluster['Segment']=='High-Value','Segment_High']=1
cluster['Segment_Mid1']=0
cluster.loc[cluster['Segment']=='Mid1-Value','Segment_Mid1']=1
cluster['Segment_Mid2']=0
cluster.loc[cluster['Segment']=='Mid2-Value','Segment_Mid2']=1
cluster['Segment_Low']=0
cluster.loc[cluster['Segment']=='Low-Value','Segment_Low']=1

In [ ]:
cluster = cluster.drop(['Segment'],axis=1)
cluster.head()

In [ ]:
# corelation with 3-month (Oct-Dec) LTVCluster
corr_matrix = cluster.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

### Building Model (lightGBM)

In [ ]:
X = cluster.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1)
y = cluster['LTVCluster']

In [ ]:
X_columns = cluster.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1).columns
y_column = ['LTVCluster']

In [ ]:
import lightgbm as lgb
from imblearn.over_sampling import SMOTE

In [ ]:
# using SMOTE for unbalanced data
X, y= SMOTE().fit_resample(X, y)

In [ ]:
# change X, y to dataframe so that we can keep column names
tX = pd.DataFrame(X, columns=X_columns)
ty = pd.DataFrame(y, columns=y_column)

In [ ]:
# take a look at y after SMOTE, 4 clusters have the same samples now
ty.hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tX, ty, test_size=0.2, random_state=56)

In [ ]:
# fitting lightGBM
params = {"objective" : "multiclass",
          "num_class": 4,
          "metric" : "multi_error",
          "num_leaves" : 30,
          "min_child_weight" : 50,
          "learning_rate" : 0.1,
          "bagging_fraction" : 0.7,
          "feature_fraction" : 0.7,
          "reg_alpha": 0.15,
          "reg_lambda": 0.15,
          "min_child_weight": 50,
          "bagging_seed" : 420,
          "verbosity" : -1
         }
lg_train = lgb.Dataset(X_train, label=y_train)
lg_test = lgb.Dataset(X_test, label=y_test)
model = lgb.train(params, lg_train, 1000, valid_sets=[lg_test], early_stopping_rounds=50, verbose_eval=100)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
# change y_pred format by select the index of maximum probability
y_pred_list = [int(np.where(i == np.amax(i))[0]) for i in y_pred]
y_pred_ar = np.array(y_pred_list)
y_pred_ar

In [ ]:
print(classification_report(y_test, y_pred_ar))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
lgb.plot_importance(model, height=0.8, ax=ax)
ax.grid(False)
plt.ylabel('Feature', size=12)
plt.xlabel('Importance', size=12)
plt.title("Importance of the Features of our LightGBM Model", fontsize=12)
plt.show()

### Adding Features

#### 1. Continent

In [ ]:
ft = df.copy()
ft["date"] = pd.to_datetime(ft["date"], format="%Y%m%d")
# select data from Jan to Sept
ft = ft[(ft.date < date(2017,10,1)) & (ft.date >= date(2017,1,1))].reset_index(drop=True)
ft.shape

In [ ]:
ft1 = ft[['fullVisitorId','geoNetwork.continent']]
ft1 = ft1.join(pd.get_dummies(ft1['geoNetwork.continent'])).drop(['(not set)'],axis = 1).drop(['geoNetwork.continent'], axis=1)

ft1 = ft1.groupby(['fullVisitorId']).agg(['max'])
ft1.columns = ['_'.join(col).strip() for col in ft1.columns.values]
ft1 = ft1.reset_index()

ft1.head()

In [ ]:
# merge continent features with cluster
ft1_df = pd.merge(cluster, ft1, on='fullVisitorId')
ft1_df.head()

In [ ]:
X = ft1_df.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1)
y = ft1_df['LTVCluster']

In [ ]:
X_columns = ft1_df.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1).columns
y_column = ['LTVCluster']

In [ ]:
# using SMOTE for unbalanced data
X, y= SMOTE().fit_resample(X, y)

In [ ]:
# change X, y to dataframe so that we can keep column names
tX = pd.DataFrame(X, columns=X_columns)
ty = pd.DataFrame(y, columns=y_column)

In [ ]:
# take a look at y after SMOTE, 4 clusters have the same samples now
ty.hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tX, ty, test_size=0.2, random_state=56)

In [ ]:
# fitting lightGBM
params = {"objective" : "multiclass",
          "num_class": 4,
          "metric" : "multi_error",
          "num_leaves" : 30,
          "min_child_weight" : 50,
          "learning_rate" : 0.1,
          "bagging_fraction" : 0.7,
          "feature_fraction" : 0.7,
          "reg_alpha": 0.15,
          "reg_lambda": 0.15,
          "min_child_weight": 50,
          "bagging_seed" : 420,
          "verbosity" : -1
         }
lg_train = lgb.Dataset(X_train, label=y_train)
lg_test = lgb.Dataset(X_test, label=y_test)
model = lgb.train(params, lg_train, 1000, valid_sets=[lg_test], early_stopping_rounds=50, verbose_eval=100)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
# change y_pred format by select the index of maximum probability
y_pred_list = [int(np.where(i == np.amax(i))[0]) for i in y_pred]
y_pred_ar = np.array(y_pred_list)
y_pred_ar

In [ ]:
print(classification_report(y_test, y_pred_ar))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
lgb.plot_importance(model, height=0.8, ax=ax)
ax.grid(False)
plt.ylabel('Feature', size=12)
plt.xlabel('Importance', size=12)
plt.title("Importance of the Features of our LightGBM Model", fontsize=12)
plt.show()

#### 2. Continent and Country-US

In [ ]:
ft2 = ft[['fullVisitorId','geoNetwork.country']]
ft2['country_US']=0
ft2.loc[ft2['geoNetwork.country']=='United States','country_US']=1
ft2 = ft2.drop(['geoNetwork.country'],axis=1)

ft2.head()

In [ ]:
# check whether a customer only stay in US/not in US, but not for both
us = ft2.groupby(['fullVisitorId']).agg(['sum'])
us.columns = ['_'.join(col).strip() for col in us.columns.values]
us = us.reset_index()

us_merge = pd.merge(user_frequency, us, on='fullVisitorId')

us_merge['us_only'] = (us_merge['Frequency']==us_merge['country_US_sum'])
us_merge.loc[us_merge['Frequency']== 1,'us_only']=True
us_merge.loc[us_merge['country_US_sum']== 0,'us_only']=True
us_merge['us_only'] = us_merge['us_only']*1

In [ ]:
# number of customer have been both in US and not in US
print(us_merge[us_merge['us_only']==0]['fullVisitorId'].count())

# presentage of customer have been both in US and not in US
print(us_merge[us_merge['us_only']==0]['fullVisitorId'].count()/us_merge.fullVisitorId.count())

Since majority of people (99.8%) will staying in US not not in US through all the trasaction, we can consider whether in US as a stable attribute for a customer.

In [ ]:
ft2 = ft2.groupby(['fullVisitorId']).agg(['max'])
ft2.columns = ['_'.join(col).strip() for col in ft2.columns.values]
ft2 = ft2.reset_index()
ft2.columns = ['fullVisitorId', 'country_US']

ft2.head()

In [ ]:
# merge continent features with cluster
ft2_df = pd.merge(ft1_df, ft2, on='fullVisitorId')
ft2_df.head()

In [ ]:
X = ft2_df.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1)
y = ft2_df['LTVCluster']

In [ ]:
X_columns = ft2_df.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1).columns
y_column = ['LTVCluster']

In [ ]:
# using SMOTE for unbalanced data
X, y= SMOTE().fit_resample(X, y)

In [ ]:
# change X, y to dataframe so that we can keep column names
tX = pd.DataFrame(X, columns=X_columns)
ty = pd.DataFrame(y, columns=y_column)

In [ ]:
# take a look at y after SMOTE, 4 clusters have the same samples now
ty.hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tX, ty, test_size=0.2, random_state=56)

In [ ]:
# fitting lightGBM
params = {"objective" : "multiclass",
          "num_class": 4,
          "metric" : "multi_error",
          "num_leaves" : 30,
          "min_child_weight" : 50,
          "learning_rate" : 0.1,
          "bagging_fraction" : 0.7,
          "feature_fraction" : 0.7,
          "reg_alpha": 0.15,
          "reg_lambda": 0.15,
          "min_child_weight": 50,
          "bagging_seed" : 420,
          "verbosity" : -1
         }
lg_train = lgb.Dataset(X_train, label=y_train)
lg_test = lgb.Dataset(X_test, label=y_test)
model = lgb.train(params, lg_train, 1000, valid_sets=[lg_test], early_stopping_rounds=50, verbose_eval=100)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
# change y_pred format by select the index of maximum probability
y_pred_list = [int(np.where(i == np.amax(i))[0]) for i in y_pred]
y_pred_ar = np.array(y_pred_list)
y_pred_ar

In [ ]:
print(classification_report(y_test, y_pred_ar))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
lgb.plot_importance(model, height=0.8, ax=ax)
ax.grid(False)
plt.ylabel('Feature', size=12)
plt.xlabel('Importance', size=12)
plt.title("Importance of the Features of our LightGBM Model", fontsize=12)
plt.show()

#### 3. Country-US Only

In [ ]:
# merge continent features with cluster
ft_us = pd.merge(cluster, ft2, on='fullVisitorId')
ft_us.head()

In [ ]:
X = ft_us.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1)
y = ft_us['LTVCluster']

In [ ]:
X_columns = ft_us.drop(['LTVCluster','logRevenue_12','fullVisitorId'],axis=1).columns
y_column = ['LTVCluster']

In [ ]:
# using SMOTE for unbalanced data
X, y= SMOTE().fit_resample(X, y)

In [ ]:
# change X, y to dataframe so that we can keep column names
tX = pd.DataFrame(X, columns=X_columns)
ty = pd.DataFrame(y, columns=y_column)

In [ ]:
# take a look at y after SMOTE, 4 clusters have the same samples now
ty.hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tX, ty, test_size=0.2, random_state=56)

In [ ]:
# fitting lightGBM
params = {"objective" : "multiclass",
          "num_class": 4,
          "metric" : "multi_error",
          "num_leaves" : 30,
          "min_child_weight" : 50,
          "learning_rate" : 0.1,
          "bagging_fraction" : 0.7,
          "feature_fraction" : 0.7,
          "reg_alpha": 0.15,
          "reg_lambda": 0.15,
          "min_child_weight": 50,
          "bagging_seed" : 420,
          "verbosity" : -1
         }
lg_train = lgb.Dataset(X_train, label=y_train)
lg_test = lgb.Dataset(X_test, label=y_test)
model = lgb.train(params, lg_train, 1000, valid_sets=[lg_test], early_stopping_rounds=50, verbose_eval=100)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
# change y_pred format by select the index of maximum probability
y_pred_list = [int(np.where(i == np.amax(i))[0]) for i in y_pred]
y_pred_ar = np.array(y_pred_list)
y_pred_ar

In [ ]:
print(classification_report(y_test, y_pred_ar))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
lgb.plot_importance(model, height=0.8, ax=ax)
ax.grid(False)
plt.ylabel('Feature', size=12)
plt.xlabel('Importance', size=12)
plt.title("Importance of the Features of our LightGBM Model", fontsize=12)
plt.show()

Adding continent and country_us has the almost the same result as adding country_us only. To avoid overfitting, we will select the third model, adding contry_us only.

In [ ]:
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
class_names=[0,1,2,3] # name  of classes

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=0)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_ar, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_ar, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()